In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collections
from nltk.corpus import stopwords
from textblob import TextBlob

In [3]:
df=pd.read_csv('yelp.csv')

basic preprocessing 

In [4]:
#convert to lowercase 
df['text'] = df['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['text'] = df['text'].str.replace('[^\w\s]','') # Remove punctuation
#Remove stop words
stop = stopwords.words('english')
stop.extend(['you','re','spa'])
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
#spelling correction
#df['text'].apply(lambda x: str(TextBlob(x).correct()))

In [5]:
yelp_reviews=df[['text','stars']]

In [6]:
yelp_reviews.head()

,text,stars
0,wife took birthday breakfast excellent weather...,5
1,idea people give bad reviews place goes show p...,5
2,love gyro plate rice good also dig candy selec...,4
3,rosie dakota love chaparral dog park convenien...,5
4,general manager scott petello good egg go deta...,5


categorize stars 1-3 as 0 anything equal to greater than 4 as 1.

In [7]:
yelp_reviews['stars'] = np.where(yelp_reviews['stars']>=4,1,0)

C:\Users\Pranay\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
texts=yelp_reviews["text"].values
labels=yelp_reviews["stars"].values

In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
import keras.utils
from keras import utils as np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

C:\Users\Pranay\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [10]:
MAX_NB_WORDS= 1000 #the maximum number of words to keep, 
                    #based on word frequency. Only the most common MAX_NB_WORDS words will be kept.



tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)


word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))



C:\Users\Pranay\Anaconda3\lib\site-packages\keras_preprocessing\text.py:175: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 36290 unique tokens.


In [11]:
MAX_SEQUENCE_LENGTH=100 # max number of words in a review to use
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = keras.utils.to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (10000, 100)
Shape of label tensor: (10000, 2)


In [12]:
# split the data into a training set and a validation set
VALIDATION_SPLIT=0.2
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [19]:
"""GLOVE_DIR='C:/Users/Pranay/Documents/GitHub/DS-Challenges/yelp data science challenge'

import os
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))"""

"GLOVE_DIR='C:/Users/Pranay/Documents/GitHub/DS-Challenges/yelp data science challenge'\n\nimport os\nembeddings_index = {}\nf = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))\nfor line in f:\n    values = line.split()\n    word = values[0]\n    coefs = np.asarray(values[1:], dtype='float32')\n    embeddings_index[word] = coefs\nf.close()\n\nprint('Found %s word vectors.' % len(embeddings_index))"

#### using RNN 

In [41]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [46]:
data=yelp_reviews
max_fatures = 1000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

In [47]:
max_fatures=1000
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 366, 128)          128000    
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 366, 128)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_13 (Dense)             (None, 2)                 394       
Total params: 383,194
Trainable params: 383,194
Non-trainable params: 0
_________________________________________________________________
None


In [48]:
Y = pd.get_dummies(data['stars']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(6700, 366) (6700, 2)
(3300, 366) (3300, 2)


In [49]:
batch_size = 100
model.fit(X_train, Y_train, epochs = 2, batch_size=batch_size)

Epoch 1/2
6700/6700 [==============================] - 292s 44ms/step - loss: 0.5695 - acc: 0.7085
Epoch 2/2
6700/6700 [==============================] - 285s 42ms/step - loss: 0.4241 - acc: 0.8116


In [50]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.43
acc: 0.80


In [51]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 85.89494163424123 %
neg_acc 66.52542372881356 %


#### Single Multi Layer Perceptron 

In [70]:
# create the model
top_words=1000
max_words=366

model = Sequential()
model.add(Embedding(top_words, 128, input_length=max_words))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 366, 128)          128000    
_________________________________________________________________
flatten_9 (Flatten)          (None, 46848)             0         
_________________________________________________________________
dense_29 (Dense)             (None, 250)               11712250  
_________________________________________________________________
dense_30 (Dense)             (None, 2)                 502       
Total params: 11,840,752
Trainable params: 11,840,752
Non-trainable params: 0
_________________________________________________________________
None


In [72]:
# Fit the model
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=2, batch_size=128, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 6700 samples, validate on 1800 samples
Epoch 1/2
 - 17s - loss: 0.2731 - acc: 0.8927 - val_loss: 0.4255 - val_acc: 0.7981
Epoch 2/2
 - 17s - loss: 0.1528 - acc: 0.9551 - val_loss: 0.4809 - val_acc: 0.7867
Accuracy: 78.67%


References:

https://www.kaggle.com/poonaml/bidirectional-lstm-spacy-on-yelp-reviews

https://machinelearningmastery.com/develop-bidirectional-lstm-sequence-classification-python-keras/

https://www.kaggle.com/bsivavenu/nlp-using-glove-and-spacy

https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

https://www.kaggle.com/ngyptr/lstm-sentiment-analysis-keras

https://machinelearningmastery.com/predict-sentiment-movie-reviews-using-deep-learning/